# Create training sub set
Create a random subset of training and val data either on a balanced set of chexpert diseases or on just one disease.

In [1]:
#hermione created: 14/07/2023
import pandas as pd
import numpy as np


In [2]:

train = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train.csv")
chexpert = pd.read_csv("/home/hermione/Documents/data/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv")
val = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid.csv")
test = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/test.csv")
train_chex = pd.merge(train, chexpert, on=['subject_id', 'study_id'])
val_chex = pd.merge(val, chexpert, on=['subject_id', 'study_id'])
test_chex = pd.merge(test, chexpert, on=['subject_id', 'study_id'])

#summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "train", "val"])
summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "train", "val","test"])
diseases = chexpert.columns[2:]

for col in diseases:
    #print(col)
    def dis_sum(data):
        return np.sum(data[data[col]==1][col].astype(int))
    """ def dis_frac(data):
        subset = data.iloc[2:]
        denom = int(sum([(subset[subset[d] == 1][d].astype(int)).sum() for d in diseases]))
        frac = dis_sum(data)/denom
        return frac """
    #summary[col] = [dis_sum(chexpert), dis_sum(train_chex), dis_sum(val_chex)]
    summary[col] = [dis_sum(chexpert), dis_sum(train_chex), dis_sum(val_chex), dis_sum(test_chex)]
    #print(np.sum(chexpert[chexpert[col]==1][col].astype(int)))
    #print(np.sum(train_chex[train_chex[col]==1][col].astype(int)))
    #print(np.sum(val_chex[val_chex[col]==1][col].astype(int)))

print(len(chexpert), len(train), len(val), len(train))
print(len(chexpert), len(train_chex), len(val_chex), len(train_chex))
#print(summary.sum(axis=1))
summary["total"] = summary.sum(axis=1)

summary #potentially make percentage


227827 33510 4894 33510
227827 33510 4894 33510


,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,total
chexpert,45808,44845,10778,27018,7179,4390,6284,51525,75455,54300,2011,16556,10358,66558,423065
train,5110,4036,856,3003,613,638,801,6631,14514,5595,260,2150,1142,6161,51510
val,753,611,104,499,70,75,111,984,2055,825,38,319,160,896,7500
test,1580,1340,276,993,191,170,236,2056,4305,1701,71,634,253,1940,15746


In [3]:

#summary_frac = pd.DataFrame(columns=summary.columns[:-1], index=summary.index)
summary_frac = []
for row in summary.index:
    columns =[]
    for col in diseases:
        def frac(data):
            return np.round(data.loc[row,col]/data.loc[row, "total"],3)
        columns.append(frac(summary))
    summary_frac.append(columns)

summary_frac_df = pd.DataFrame(summary_frac ,columns=summary.columns[:-1], index=summary.index)
summary_frac_df

,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
chexpert,0.108,0.106,0.025,0.064,0.017,0.010,0.015,0.122,0.178,0.128,0.005,0.039,0.024,0.157
train,0.099,0.078,0.017,0.058,0.012,0.012,0.016,0.129,0.282,0.109,0.005,0.042,0.022,0.120
val,0.100,0.081,0.014,0.067,0.009,0.010,0.015,0.131,0.274,0.110,0.005,0.043,0.021,0.119
test,0.100,0.085,0.018,0.063,0.012,0.011,0.015,0.131,0.273,0.108,0.005,0.040,0.016,0.123


In [3]:
#select data
train_data_index = train_chex[train_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
val_data_index = val_chex[val_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
train_data = train_chex[train_chex.index.isin(train_data_index)| (train_chex["Pleural Effusion"] == 1)]

val_data = val_chex[val_chex.index.isin(val_data_index) | (val_chex["Pleural Effusion"] == 1)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 10433
10433 1510


In [4]:
train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_subset.csv",index=False)
val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_subset.csv", index=False)

In [6]:
#select data
train_data_index = train_chex[train_chex["No Finding"] == 1].sample(n=10000, random_state=1).index
val_data_index = val_chex[val_chex["No Finding"] == 1].sample(n=1500, random_state=1).index

train_data = train_chex[~train_chex.index.isin(train_data_index)]
val_data = val_chex[~val_chex.index.isin(val_data_index)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

#train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_nf_eq.csv",index=False)
#val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_nf_eq.csv", index=False)

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 23510
23510 3394


In [7]:
#definitly not overlapping
print(train_final[train_final["mimic_image_file_path"].isin(val_final["mimic_image_file_path"])])

Empty DataFrame
Columns: [subject_id, study_id, image_id, mimic_image_file_path, reference_report]
Index: []


# Remake Datasets from bounding box reports
This shouls hopefully only keep pathology relevant line rather than scan veiws or many references to previos scans. 


In [32]:
from ast import literal_eval
train_bb = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports_original/train.csv").drop(columns=["bbox_coordinates","bbox_labels","bbox_is_abnormal", "bbox_phrase_exists"])
train_bb["reference_report"] = [" ".join(" ".join(list(set(literal_eval(phrases)))).split()) for phrases in train_bb["bbox_phrases"]]


In [33]:

train_bb = train_bb.drop(columns=["bbox_phrases"])
print(len(train_bb))
val_bb = train_bb.sample(frac=0.1, random_state=1)
train_bb = train_bb[~train_bb["image_id"].isin(val_bb["image_id"])]
test_bb = train_bb.sample(n=len(val_bb), random_state=1)
train_bb = train_bb[~train_bb["image_id"].isin(test_bb["image_id"])]
val_bb

49311


,subject_id,study_id,image_id,mimic_image_file_path,reference_report
18647,11172056,50366239,324f2928-3d040888-70b3742c-f2c3b988-63cc7617,/home/hermione/Documents/data/physionet.org/fi...,There has been interval resolution of the bord...
48959,10766641,55573119,abbfc4d2-4c08ecd5-95216c83-f9e7e8d4-f0e03a44,/home/hermione/Documents/data/physionet.org/fi...,No focal consolidation or pleural effusion. Th...
22567,11400517,56972034,f60572ea-5c0e131d-0be53637-2333841e-2c930ea3,/home/hermione/Documents/data/physionet.org/fi...,No pleural effusion or pneumothorax is seen. T...
35385,12144619,58964820,bf5c0d9f-b4cb1207-607aacb3-f6a24029-04dec268,/home/hermione/Documents/data/physionet.org/fi...,"As compared to the previous radiograph, the lu..."
34886,12124186,52107348,dbf88c6a-071650e5-0ec82dc7-b4f58f7f-54c13ee5,/home/hermione/Documents/data/physionet.org/fi...,"Normal heart size, mediastinal and hilar conto..."
...,...,...,...,...,...
21826,11346699,50584318,617afa71-673719dc-0dbecb50-028de5c8-38e3434e,/home/hermione/Documents/data/physionet.org/fi...,The degree of pulmonary edema appears to be sl...
14664,10922531,51051513,8646ffe3-5b8b6570-bbc7fb58-c8489fe0-9af4b9ac,/home/hermione/Documents/data/physionet.org/fi...,"I presume that these are in the pleural space,..."
9265,10584015,51857744,ceca7221-1f04d4e0-e3048c54-6671d2f4-5faf37e6,/home/hermione/Documents/data/physionet.org/fi...,Borderline size of the cardiac silhouette with...
5520,10353355,53929101,397b0512-87c35ce7-490c463c-51fc61fb-07eb75e6,/home/hermione/Documents/data/physionet.org/fi...,No pleural effusion or pneumothorax is present...


In [30]:
print(len(train_bb), len(val_bb), len(test_bb))
print(len(train), len(val), len(test))

39449 4931 4931
33510 4894 10147


In [34]:
print(train_bb["reference_report"][0])
#print(train_bb["bbox_phrases"][0])
train_bb

Heart is top-normal size, improved, and pleural effusions have resolved. Previous mild pulmonary edema and possible concurrent pneumonia has all cleared. Right hilar vessels are still enlarged, perhaps due to pulmonary arterial hypertension. Previous mild pulmonary edema and possible concurrent pneumonia has all cleared. Previous mild pulmonary edema and possible concurrent pneumonia has all cleared. Heart is top-normal size, improved, and pleural effusions have resolved. Right hilar vessels are still enlarged, perhaps due to pulmonary arterial hypertension. Lateral view shows atherosclerotic coronary calcification in the left circumflex. Heart is top-normal size, improved, and pleural effusions have resolved. Lateral view shows atherosclerotic coronary calcification in the left circumflex. Previous mild pulmonary edema and possible concurrent pneumonia has all cleared. Heart is top-normal size, improved, and pleural effusions have resolved.


,subject_id,study_id,image_id,mimic_image_file_path,reference_report
0,10000980,50985099,6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d,/home/hermione/Documents/data/physionet.org/fi...,"Heart is top-normal size, improved, and pleura..."
1,10000980,51967283,943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4,/home/hermione/Documents/data/physionet.org/fi...,Right upper lobe pneumonia or mass. The remain...
2,10000980,54577367,cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8,/home/hermione/Documents/data/physionet.org/fi...,The aorta is calcified and diffusely tortuous....
3,10000980,54935705,6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a,/home/hermione/Documents/data/physionet.org/fi...,There are possible small bilateral pleural eff...
4,10000980,54980801,a75a1fbe-802065ad-717eb7c1-e2ce3552-646276a6,/home/hermione/Documents/data/physionet.org/fi...,"The lungs are clear of consolidation, effusion..."
...,...,...,...,...,...
49305,12949794,54324761,c705f23c-5e110086-92d63fed-ce2390ab-0aaab539,/home/hermione/Documents/data/physionet.org/fi...,Lateral view shows the PICC tip in the lower S...
49307,12958472,57693298,ef33ed50-3103c924-869900f8-dfb73dd3-8b1f0c0a,/home/hermione/Documents/data/physionet.org/fi...,Moderate right pleural effusion and multifocal...
49308,12963147,56053808,665650b7-273d4330-d61f391c-8a4663ab-22ae3715,/home/hermione/Documents/data/physionet.org/fi...,Lungs are low in volume resulting in bronchova...
49309,12989742,59169029,d072a686-274dbd4a-aa1d2b90-61ba527b-d4b9ebdd,/home/hermione/Documents/data/physionet.org/fi...,Cardiomediastinal silhouette appears normal. N...
